## General Concatenate guidelines:

Fasta files must be aligned, can be nucelotide, AA or both, or neither
In each alignment: all genomes must have the same name across all files, and only 1 seq
    Duplicates, paralogues, short matches must be removed, or else introduced as another "genome"

## Using FasConCatG

In [ ]:
perl /path/Fasconcatg_v1.04.pl -s -l
# in a directory with your alignments

In [1]:
import sys

In [2]:
sys.version

'2.7.17 (default, Sep 30 2020, 13:38:04) \n[GCC 7.5.0]'

#file extension matters only accepts: fasta, fas, nex, phy, but can output to either.
#l command gives your partition file (RaxML), can also set the output format to nexus (Iqtree)
#missing data and gaps, etc in info.xls

Also a bunch of other features: consensus seq, omit certain codon positions, file format converter, recoding (ie Dayhoff).

In [ ]:
#nexus  <-[make sure this is actually at the beginning (not a comment)]
begin sets;
    charset part1 = 1-100;
    charset part2 = 101-384;
    charpartition mine = HKY+G:part1, GTR+I+G:part2; # this one is optional if you don't know the best models yet
end;

##  Wrapper Supermatrix

In [ ]:
# same set up as before, to create your A list:
ls *.fas > Alist.txt
supermatrix_wrapper.py Alist.txt
#take the last outfile for the full alignmnet
#you have to cat *.paritions and remove some lines via regex

## C60 Model - IqTree
A mixture model with 60 defined classes, takes a long time (on its own)
Note: a mixture model does not need partitions just throw in the supermatrix and wait (if using the models on IQtree, it only takes a protein alignment)

### What is the difference between partition and mixturemodels?
Mixture models, like partition models, allow more than one substitution model alongthe sequences. However, while a partition model assigns each alignment site a givenspecific model, mixture models do not need this information. A mixture model willcompute for each site its probability (or weight) of belonging to each of the mixtureclasses (also called categories or components). Since the site-to-class assignment is un-known, the site likelihood under mixture models is the weighted sum of site likelihoodsper mixture class.For example, thediscrete Gamma rate heterogeneityis a simple mixture model type.It has several rate categories with equal weight. IQ-TREE also supports a number ofpredefined protein mixture modelssuch as the profile mixture modelsC10toC60(TheML variants of BayesianCATmodels)

In [ ]:
## Paritions




iqtree -s alignment.fasta -nt AUTO -m MFP+MERGE -spp parition.nex # slowest, merges like partitions -bb 1000

iqtree -s alignment.fasta -nt AUTo -m MF+MERGE -spp partition.nex -rcluster 10 #faster, only considers the top 10% of models and merging schemes

iqttree -s alignment.fasta -nt AUTO -m MF+MERGE -spp partition.nex -rcluster 10 -mset LG,WAG # fastests, only top 10% within LG, WAG (can change with what ever model set)

In [ ]:
# Mixture
#easiest, but longest, and closest to Bayesian
iqtree -s alignment.fasta -nt AUTO -m LG+C60 -bb 1000 -wbt -wsl -wsr

#fastest (like 1/10 of the time) and almost just as good (sometimes called 'cat' or 'PMSF' (posterior mean site frequency))
iqtree -s alignment.fasta -nt AUTO -m LG+F+G -bb 1000 -wbt -wsl -wsr
# we use a site homogenous model (LG+F+G) to calculate a guide tree (which is fast)
# then feed the guide tree back into LG+C*
iqtree -s alignment.fasta -ft treefile(from last command) -nt AUTO -m LG+C60 -bb 1000 -wbt -wsl -wsr



## Other supermatrix things
### Fast evolving sites
The -wsr -wsl writes the site log likelihoods and site rates to a file which you can feed to sitestripper.pl to removed fast evolving sites. (ex: keep 70% of sites) 
### Recoding
Dayhoff4, Dayhoff6 on proteins work in phylobayes but also takes a lonnnnng time (and chains may never converge)
For dayhoff4 -> code into nucleotides [A,G,C,T], use the user defined model for C60 in iqtree ( can also use PMSF).
### Concordance
(http://www.iqtree.org/doc/Concordance-Factor)
essentially you build a tree for every partition then you can map the topologies of every tree onto any reference tree as supports
### Topology Tests
-wbt prints all the bootstrap trees into a file so you have a sample of trees corresponding to a dataset that can be used for -au -sh etc tests